#### Consolidación del dataset para el desarrollo de consultas

- Para desarrollar la API descrita en la propuesta de trabajo (README.md) se hizo necesario para optimizar recursos de computo y memoria consolidar un solo dataset para responder las consultas y los endpoints del proyecto.

- Es por esto que, considerando las columnas que se deben consultar y optimizando el tamaño del dataset para fines demostrativos en este proyecto se consolida este dataset.

- Como lo anuncia la propuesta de trabajo, hemos resumido el análisis de sentimiento en una nueva columna sentiment que contiene si la reseña que tiene el juego es positiva, neutral o negativa. Seguiremos trabajando para los endpoints usando exclusivamente estos juegos con reseñas para que la muestra de nuestro proyecto este orientada a esos juegos que fueron calificados y que nos interesa conocer por sus desarrolladores y las recomendaciones de los usuarios.

- Se harán varios 'merge' de los datasets anteriores para alcanzar lo anteriormente descrito, primero se unirá el dataset de reseñas (users_review_df) con el dataset de usuarios y juegos (users_items_df) y posteriormente se fusionará con el dataset de juegos de steam (steam_games_df). 

In [25]:
import pandas as pd

# Carga de los archivos
file_path_games = '../data/processed/processed_steam_games.csv'
steam_games_df = pd.read_csv(file_path_games)

file_path_reviews = '../data/processed/processed_users_reviews.csv'
users_reviews_df = pd.read_csv(file_path_reviews)

file_path_user_items = '../data/processed/processed_users_items.csv'
users_items_df = pd.read_csv(file_path_user_items)

# Uniendo users_items_df con users_reviews_df
combined_reviews_items_df = pd.merge(users_reviews_df, users_items_df, on=['user_id', 'item_id'], how='inner')

# Cambiando el nombre de la columna para el merge con steam_games_df
steam_games_df.rename(columns={'id': 'item_id'}, inplace=True)

# Combinando con steam_games_df
df_complete = pd.merge(combined_reviews_items_df, steam_games_df, on='item_id', how='left')

# Reemplaza los valores NaN de la columna sentiment con la opción por defecto 1: neutral (si es necesario)
df_complete['sentiment'] = df_complete['sentiment'].fillna(1)

# Mostrar el DataFrame resultante
df_complete

,user_id,item_id,sentiment,items_count,playtime_forever,genres,app_name,price,developer,release_year
0,76561197970982479,1250,1,277,10006.0,Action,Killing Floor,19.99,Tripwire Interactive,2009.0
1,js41637,251610,1,888,84.0,NaN,NaN,NaN,NaN,NaN
2,evcentric,248820,2,137,2328.0,Action,Risk of Rain,9.99,"Hopoo Games, LLC",2013.0
3,evcentric,248820,2,137,2328.0,Indie,Risk of Rain,9.99,"Hopoo Games, LLC",2013.0
4,evcentric,248820,2,137,2328.0,RPG,Risk of Rain,9.99,"Hopoo Games, LLC",2013.0
...,...,...,...,...,...,...,...,...,...,...
39057,sexyawp,427730,1,9,101.0,Indie,Who's Your Daddy,4.99,Evil Tortilla Games,2015.0
39058,76561198296402247,280790,1,11,2984.0,NaN,NaN,NaN,NaN,NaN
39059,Ghoustik,730,1,4,3969.0,Action,Counter-Strike: Global Offensive,14.99,Valve,2012.0
39060,76561198312638244,233270,2,36,533.0,Action,Far Cry 3 - Blood Dragon,14.99,Ubisoft Montreal,2013.0


Luego de consolidar nuestro dataset, vamos a eliminar los registros duplicados si existen y hacernos cargos de registros con valores nulos en géneros, año de lanzamiento y horas totales de juego que son columnas necesarias para las consultas.

In [26]:
# Verificamos los valores nulos resultantes
df_complete.isna().sum()

user_id                0
item_id                0
sentiment              0
items_count            0
playtime_forever       0
genres              2258
app_name            2258
price               2521
developer           2258
release_year        2258
dtype: int64

In [27]:
# Manejo de valores NaN
df_complete = df_complete.dropna(subset=['genres', 'release_year', 'playtime_forever'])

# los precios nulos los dejaremos en cero
df_complete['price'] =  df_complete['price'].fillna(0)

/var/folders/j1/pdsb5w0x52sgplylm6f7fmgm0000gn/T/ipykernel_75441/1261996023.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_complete['price'] =  df_complete['price'].fillna(0)


In [28]:
# Verificamos valores nulos
df_complete.isna().sum()

user_id             0
item_id             0
sentiment           0
items_count         0
playtime_forever    0
genres              0
app_name            0
price               0
developer           0
release_year        0
dtype: int64

In [30]:
# Vemos los valores únicos de la columna 'price' para verificar que quede como variable numérica
df_complete['price'].unique()

array(['19.99', '9.99', '24.99', '14.99', '29.99', 'Free to Play', '4.99',
       '34.99', '17.99', '2.99', '1.99', '7.99', '11.99', 'Free', '3.99',
       '59.99', '49.99', '39.99', '0.99', 'Free To Play', '12.99',
       '59.95', '6.99', 'Play for Free!', 0, '5.99', '44.99', '13.98',
       '13.99', '8.99', 'Free HITMAN™ Holiday Pack', '54.99', '13.37',
       '18.99', '20.0', '160.91', '74.76', '16.99', '139.92', 'Free Mod',
       '79.99', '16.06', '15.99', '69.99', 'Free to Use', '7.49', '10.99',
       '0.49', '23.99', '21.99', '189.96', 'Third-party'], dtype=object)

In [31]:
# Reemplazamos valores de price que sean cadenas de strings
df_complete['price'] = df_complete['price'].replace({"Free To Play": 0, "Play for Free!": 0, "Free Mod": 0, "Free HITMAN™ Holiday Pack": 0,
                           "Free to Use": 0, "Third-party": 0, "Free to Play": 0, "Free":0})

df_complete['price'].value_counts()

price
0         9838
19.99     6228
14.99     5535
9.99      5386
34.99     1277
39.99     1156
4.99      1113
29.99     1049
59.99      919
24.99      531
0.99       416
2.99       297
7.99       232
12.99      175
1.99       156
6.99       151
5.99       131
3.99       126
49.99      115
17.99       91
11.99       89
13.98       48
20.0        45
8.99        38
59.95       24
13.99       24
74.76       24
160.91      20
16.99       13
139.92      12
44.99       12
16.06       10
18.99        7
15.99        7
13.37        6
69.99        6
0.49         6
189.96       6
54.99        4
10.99        4
23.99        3
21.99        3
79.99        2
7.49         1
Name: count, dtype: int64

In [34]:
# Eliminación de duplicados
df_complete = df_complete.drop_duplicates()

df_complete

,user_id,item_id,sentiment,items_count,playtime_forever,genres,app_name,price,developer,release_year
0,76561197970982479,1250,1,277,10006.0,Action,Killing Floor,19.99,Tripwire Interactive,2009.0
2,evcentric,248820,2,137,2328.0,Action,Risk of Rain,9.99,"Hopoo Games, LLC",2013.0
3,evcentric,248820,2,137,2328.0,Indie,Risk of Rain,9.99,"Hopoo Games, LLC",2013.0
4,evcentric,248820,2,137,2328.0,RPG,Risk of Rain,9.99,"Hopoo Games, LLC",2013.0
5,doctr,250320,2,541,778.0,Adventure,The Wolf Among Us,24.99,Telltale Games,2013.0
...,...,...,...,...,...,...,...,...,...,...
39056,sexyawp,427730,1,9,101.0,Casual,Who's Your Daddy,4.99,Evil Tortilla Games,2015.0
39057,sexyawp,427730,1,9,101.0,Indie,Who's Your Daddy,4.99,Evil Tortilla Games,2015.0
39059,Ghoustik,730,1,4,3969.0,Action,Counter-Strike: Global Offensive,14.99,Valve,2012.0
39060,76561198312638244,233270,2,36,533.0,Action,Far Cry 3 - Blood Dragon,14.99,Ubisoft Montreal,2013.0


Ya con el dataset depurado, lo guardaremos para que sea el dataset definitivo para las 5 primeras consultas de la API

In [35]:
# Guardamos el dataframe procesado para las consultas de la API
output_file_path = '../src/data/dataset_full.csv'
df_complete.to_csv(output_file_path, index=False)

output_file_path

'../src/data/dataset_full.csv'

El desarrollo de las funciones para las consultas se realiza en el archivo funciones.py que posteriormente son usadas en el main.py